In [5]:
import plotly.graph_objects as go
from itertools import product
import networkx as nx
from matplotlib import pyplot as plt

# G = nx.random_geometric_graph(200, 0.125)

In [6]:
import yaml
with open('website_by_type.yaml', 'r') as stream:
    website_by_type = yaml.safe_load(stream)

import pickle
with open('web2tracker_domains.pkl', 'rb') as handle:
    web2domain = pickle.load(handle)
print(web2domain)

defaultdict(<class 'set'>, {'Google.com': {'Google'}, 'Youtube.com': {'Google'}, 'Amazon.com': {'Amazon', 'Facebook', 'Twitter', 'OpenX'}, 'Facebook.com': {'Facebook', 'Google'}, 'Yahoo.com': {'Facebook', 'comScore', 'Rubicon Project', 'PubMatic', 'Yahoo', 'Pubmatic', 'Index Exchange', 'Google', 'AppNexus', 'Amazon', 'Verizon'}, 'Reddit.com': {'Reddit', 'comScore', 'Flashtalking', 'Quantcast', 'A.A.Exchange', 'Kantar', 'Acxiom', 'Google', 'Yahoo', 'Amazon', 'AAX', 'A.A.Ex.'}, 'Ebay.com': {'Ebay', 'eBay', 'ThreatMatrix', 'Google', 'ThreatMetrix'}, 'Netflix.com': {'Facebook', 'Google', 'Netflix'}, 'Live.com': {'Optimizely', 'Microsoft'}, 'Twitch.tv': {'Salesforce', 'Branch', 'comScore', 'Branch Metrics', 'Google', 'Amazon', 'Nielsen'}, 'Myshopify.com': {'Nielsen'}, 'Instructure.com': {'Facebook', 'Marketo', 'Akamai', 'Twitter', 'Lucky Orange', 'DemandBase', 'Trade Desk', 'Demandbase', 'Wingify', 'LiveChat', 'Beeswax', 'Crazy Egg', 'Microsoft', 'Wistia', 'AppDynamics', 'New Relic', 'Tribl

In [21]:
print('1', web2domain['Adobe.com'])
print('2', web2domain['Spotify.com'])

1 {'Facebook', 'Demandbase', 'Evidon', 'iPerceptions', 'Flashtalking', 'Twitter', 'Adobe', 'Microsoft', 'Google', 'OpenX', 'Akamai', 'MediaMath', 'WellsFargo'}
2 {'Facebook', 'VisualIQ', 'comScore', 'Twitter', 'Adobe', 'Hotjar', 'Snapchat', 'Hot Jar', 'Acxiom', 'Google', 'Fastly', 'Pinterest', 'Telenor', 'Oracle', 'Trade Desk', 'Nielsen'}


In [18]:
# website_white_list = "" # = set(website_by_type["News"])
website_black_list = {"Google.com"}

G = nx.Graph()
for x in web2domain.keys():
    for y in web2domain[x]:
        y_domain_name = y + '.com' if '.com' not in y else y
        if y_domain_name != x and y_domain_name in web2domain.keys() and y_domain_name not in website_black_list:
            G.add_node(x)
            G.add_node(y_domain_name)
            G.add_edge(x, y_domain_name)

# nx.draw(G, with_labels=True)
# plt.show()
# plt.savefig("preliminary_figure.png")
pos = nx.random_layout(G)
# pos
print(G.edges)

[('Amazon.com', 'Facebook.com'), ('Amazon.com', 'Twitter.com'), ('Amazon.com', 'Yahoo.com'), ('Amazon.com', 'Reddit.com'), ('Amazon.com', 'Twitch.tv'), ('Amazon.com', 'Cnn.com'), ('Amazon.com', 'Espn.com'), ('Amazon.com', 'Nytimes.com'), ('Amazon.com', 'Intuit.com'), ('Amazon.com', 'Imdb.com'), ('Amazon.com', 'Amazonaws.com'), ('Amazon.com', 'Imgur.com'), ('Facebook.com', 'Yahoo.com'), ('Facebook.com', 'Netflix.com'), ('Facebook.com', 'Instructure.com'), ('Facebook.com', 'Cnn.com'), ('Facebook.com', 'Instagram.com'), ('Facebook.com', 'Linkedin.com'), ('Facebook.com', 'Chase.com'), ('Facebook.com', 'Dropbox.com'), ('Facebook.com', 'Intuit.com'), ('Facebook.com', 'Adobe.com'), ('Facebook.com', 'Zillow.com'), ('Facebook.com', 'Wellsfargo.com'), ('Facebook.com', 'Spotify.com'), ('Facebook.com', 'Aliexpress.com'), ('Facebook.com', 'Walmart.com'), ('Facebook.com', 'Hulu.com'), ('Facebook.com', 'Paypal.com'), ('Facebook.com', 'Imgur.com'), ('Twitter.com', 'Instructure.com'), ('Twitter.com', '

In [15]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node_name in pos.keys():
    x, y = pos[node_name]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2),
    name="marker and text",
    text=list(pos.keys()),
    textposition="bottom center")


In [16]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
# node_trace.text = node_text

In [17]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Domain Sharing among Alexa Top 50 website',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                # annotations=[ dict(
                #     text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                #     showarrow=False,
                #     xref="paper", yref="paper",
                #     x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()